# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities_jimmy.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,-7.21,77,21,5.05,GS,1717988709
1,1,saipan,15.1355,145.7010,33.39,66,40,8.23,MP,1717988709
2,2,jianshui,23.6363,102.8264,24.61,77,85,1.79,CN,1717988710
3,3,lebu,-37.6167,-73.6500,12.95,86,94,8.79,CL,1717988710
4,4,limassol,34.6750,33.0333,27.32,78,0,0.51,CY,1717988669


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat',
                                      geo=True, 
                                      tiles='OSM', 
                                      color='City', 
                                      size='Humidity', 
                                      hover_cols=['City'],
                                      frame_height=400, frame_width=600)

# Display the map
map_plot



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df[(city_data_df['Cloudiness'] <= 50) &
                                  (city_data_df['Max Temp'] < 25) &
                                  (city_data_df['Humidity'] <= 60) &
                                  (city_data_df['Wind Speed'] < 9)]


# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
11,11,port alfred,-33.5906,26.8910,15.37,59,13,3.02,ZA,1717988712
28,28,udachny,66.4167,112.4000,21.36,32,33,4.05,RU,1717988716
29,29,flagstaff,35.1981,-111.6513,22.81,25,0,1.54,US,1717988657
33,33,thompson,55.7435,-97.8558,18.09,29,0,2.57,CA,1717988717
40,40,east london,-33.0153,27.9116,14.64,52,5,2.50,ZA,1717988719


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
11,port alfred,ZA,-33.5906,26.8910,59,
28,udachny,RU,66.4167,112.4000,32,
29,flagstaff,US,35.1981,-111.6513,25,
33,thompson,CA,55.7435,-97.8558,29,
40,east london,ZA,-33.0153,27.9116,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port alfred - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
flagstaff - nearest hotel: Residence Inn
thompson - nearest hotel: Thompson Inn
east london - nearest hotel: No hotel found
port macquarie - nearest hotel: The Observatory
orkney - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
qamdo - nearest hotel: 银杏商务宾馆
yellowknife - nearest hotel: No hotel found
altai - nearest hotel: Altai hotel 40000-100000T
yasuj - nearest hotel: هتل ارم
dryden - nearest hotel: Holiday Inn Express
isfahan - nearest hotel: هتل فیروزه
awjilah - nearest hotel: No hotel found
ak-suu - nearest hotel: BellavistaHotel
belyy yar - nearest hotel: No hotel found
tindouf - nearest hotel: محمد بوسبي
luau - nearest hotel: No hotel found
chilliwack - nearest hotel: The Royal Hotel
tsetserleg - nearest hotel: Буян зочид буудал
upington - nearest hotel: Die Eiland Holiday Ressort
villazon - nearest hotel: Hotel Ideal
karagandy - nearest hotel: Аппартамент

,City,Country,Lat,Lng,Humidity,Hotel Name
11,port alfred,ZA,-33.5906,26.8910,59,No hotel found
28,udachny,RU,66.4167,112.4000,32,Вилюй
29,flagstaff,US,35.1981,-111.6513,25,Residence Inn
33,thompson,CA,55.7435,-97.8558,29,Thompson Inn
40,east london,ZA,-33.0153,27.9116,52,No hotel found
...,...,...,...,...,...,...
529,heihe,CN,50.2441,127.4902,36,Санкт-Петербург
538,posse,BR,-14.0931,-46.3694,56,Palmeiras Hotel
550,jalai nur,CN,49.4500,117.7000,36,满洲里福门客栈
551,urangan,AU,-25.3000,152.9000,50,Ramada


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points('Lng', 'Lat',
                              hover_cols=['City','Humidity','Hotel Name','Country'],
                              geo=True,
                              tiles='OSM',
                              color='City',
                              size = 'Humidity',
                              frame_height=400, frame_width=600)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)